# Air Pollution by Politics

## Imports & Installs

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import (
    r2_score, mean_squared_error, explained_variance_score,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC

from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy.signal import argrelextrema
from kneed import KneeLocator

sys.path.append('../etc')
from refs import state_abbreviation_dict

# Optional (only if you truly need them at runtime)
!pip install kneed
!pip install -q xlrd openpyxl

## Loads

Since the `data` folder is approximately **14 GB**, it has been uploaded to a **shared Google Drive folder**.
To access and use this data in this notebook, follow these steps:

---

### 1. Upload this notebook to **Google Colab**
- Open [Google Colab](https://colab.research.google.com/).
- Click on **File → Upload notebook** and select this notebook.

### 2. Mount your Google Drive
Insert and run the following code in a new Colab cell:

```python
from google.colab import drive
drive.mount('/content/drive')
```
- This will prompt you to authorize access to your Google Drive.

### 3. Create a Shortcut of the `data` Folder in Your Google Drive
- Open the shared Google Drive link.
- **Right-click** the `data` folder → **Add shortcut to Drive**.
- Choose a location inside your **MyDrive** (e.g., directly under "MyDrive" or inside a project folder).

### 4. Update the File Paths in the Notebook
When reading the data files, adjust the paths in your code to point to your shortcut location.
For example:

```python
import pandas as pd

# Example
presidential_data = pd.read_csv('/content/drive/MyDrive/your_folder_name/data/election/relevant/usa/1976-2020-president.csv')
```

Replace `your_folder_name` with the actual folder name where you added the shortcut.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# presidential_data = pd.read_csv('/content/drive/MyDrive/School/data/election/relevant/usa/1976-2020-president.csv')
# senate_data = pd.read_csv('/content/drive/MyDrive/School/data/election/relevant/usa/1976-2020-senate.csv')
# house_data = pd.read_csv('/content/drive/MyDrive/School/data/election/relevant/usa/1976-2022-house.csv')
# house_data['party_simplified'] = house_data['party'].apply(lambda x: x if x in ['DEMOCRAT', 'REPUBLICAN'] else 'OTHER')
#
# state_emissions_data = pd.read_excel(
#     "/content/drive/MyDrive/School/data/co2_emissions/relevant/1970-2022-state_emissions.xlsx",
#     engine="openpyxl",
#     skiprows=4
# )
#
# state_population_size_data = pd.read_csv('/content/drive/MyDrive/School/data/general_state_data/population_data.csv')
# state_population_size_data.rename(columns={'AK': 'state', '1950': 'year', '135000': 'population'}, inplace=True)
# state_land_size_data = pd.read_csv('/content/drive/MyDrive/School/data/general_state_data/state_sizes.csv')

In [ ]:
presidential_data = pd.read_csv('../data/election/relevant/usa/1976-2020-president.csv')
senate_data = pd.read_csv('../data/election/relevant/usa/1976-2020-senate.csv')
house_data = pd.read_csv('../data/election/relevant/usa/1976-2022-house.csv')
house_data['party_simplified'] = house_data['party'].apply(lambda x: x if x in ['DEMOCRAT', 'REPUBLICAN'] else 'OTHER')

state_emissions_data = pd.read_excel(
    "../data/co2_emissions/relevant/1970-2022-state_emissions.xlsx",
    engine="openpyxl",
    skiprows=4
)

state_population_size_data = pd.read_csv('../data/general_state_data/population_data.csv')
state_population_size_data.rename(columns={'AK': 'state', '1950': 'year', '135000': 'population'}, inplace=True)
state_land_size_data = pd.read_csv('../data/general_state_data/state_sizes.csv')

##  Analysis Per Category

### Election Analysis

This code calculates a political score for each U.S. state based on presidential, House, and Senate election data. It processes the election data for each year, calculates political scores by considering factors like vote percentage, longest streak of party wins, and the party of the most recent winner. It then combines these scores using weighted averages for each state and year. The final political scores are scaled and adjusted using the `MinMaxScaler`, and the scores are returned in a DataFrame, where each row represents a state’s political score for a specific year. This method provides insights into state-level political trends over time.

In [ ]:
def calculate_scores_by_year(presidential_data, house_data, senate_data):
    years = sorted(presidential_data['year'].unique())
    all_scores = []

    for year in years:
        pres_results = calculate_political_scores(presidential_data, year)
        house_results = calculate_political_scores(house_data, year)
        sen_results = calculate_political_scores(senate_data, year)
        final_political_score = merge_and_average_scores(house_results, sen_results, pres_results, year)
        all_scores.append(final_political_score)

    return pd.concat(all_scores).reset_index(drop=True)

def calculate_political_scores(df, year=2020):
    df = df[df['year'] <= year].copy()  # Ensure a full copy is made
    df_filtered = df[df['party_simplified'].isin(['DEMOCRAT', 'REPUBLICAN'])].copy()

    df_filtered.loc[:, 'vote_percent'] = df_filtered['candidatevotes'] / df_filtered['totalvotes'] * 100

    df_winner = df_filtered.loc[df_filtered.groupby(['year', 'state'])['candidatevotes'].idxmax()].copy()

    state_streaks = {}
    for state in df_winner['state'].unique():
        state_data = df_winner[df_winner['state'] == state].sort_values('year')
        current_party = None
        streak = 0
        max_streak = 0
        last_winner = None

        for _, row in state_data.iterrows():
            if row['party_simplified'] == current_party:
                streak += 1
            else:
                current_party = row['party_simplified']
                streak = 1
            max_streak = max(max_streak, streak)
            last_winner = current_party

        state_streaks[state] = {'longest_streak': max_streak, 'last_winner': last_winner}

    df_summary = df_winner.groupby('state').agg(
        total_elections=('year', 'count'),
        avg_vote_percent=('vote_percent', 'mean'),
        last_winner=('party_simplified', 'last'),
        last_vote_percent=('vote_percent', 'last')
    ).reset_index()

    df_summary['longest_streak'] = df_summary['state'].map(lambda x: state_streaks[x]['longest_streak'])
    df_summary['streak_party'] = df_summary['state'].map(lambda x: state_streaks[x]['last_winner'])

    df_summary['party_numeric'] = df_summary['last_winner'].map({'REPUBLICAN': -1, 'DEMOCRAT': 1})

    features = df_summary[['longest_streak', 'avg_vote_percent', 'last_vote_percent']]

    scaler = MinMaxScaler(feature_range=(-1, 1))
    df_summary.loc[:, ['streak_score', 'avg_vote_score', 'last_vote_score']] = scaler.fit_transform(features)

    df_summary['final_score'] = (df_summary['streak_score'] * 0.4 +
                                 df_summary['avg_vote_score'] * 0.3 +
                                 df_summary['last_vote_score'] * 0.3) * df_summary['party_numeric']
    df_summary['year'] = year
    return df_summary[['state', 'year', 'final_score']]

def merge_and_average_scores(house_results, sen_results, pres_results, year=2020):
    df_combined = house_results.merge(sen_results, on=['state', 'year'], suffixes=('_house', '_sen')) \
                               .merge(pres_results, on=['state', 'year'], suffixes=('', '_pres'))

    df_combined['final_score_avg'] = df_combined[['final_score_house', 'final_score_sen', 'final_score']].mean(axis=1)

    return df_combined[['state', 'year', 'final_score_avg']]

final_political_scores_by_year = calculate_scores_by_year(presidential_data, house_data, senate_data)
final_political_scores_by_year = final_political_scores_by_year.rename(columns={'final_score_avg':'party_affiliation_score'})
final_political_scores_by_year

### Emissions Analysis

This code analyzes state-level emissions data by excluding any rows containing "Total" in the state name. It then calculates the average emissions for each state across the available years and identifies the top 10 states with the highest and lowest average emissions. The results are visualized in two separate bar charts: one for the top 10 states with the highest emissions, and another for the top 10 states with the lowest emissions, using red and green colors, respectively. These plots help to easily identify and compare the states based on their average emissions over time.

In [ ]:
state_emissions_data

In [ ]:
state_emissions_data = state_emissions_data[~state_emissions_data['State'].str.contains("Total", case=False, na=False)]
year_columns = [col for col in state_emissions_data.columns if isinstance(col, int)]
top_states = state_emissions_data.set_index('State')[year_columns].mean(axis=1).nlargest(10)
bottom_states = state_emissions_data.set_index('State')[year_columns].mean(axis=1).nsmallest(10)

plt.figure(figsize=(12, 6))
top_states.plot(kind='bar', color='red', alpha=0.7)
plt.title("Top 10 States with Highest Yearly Average Emissions(measured in million metric tons of carbon dioxide)")
plt.xlabel("State")
plt.ylabel("Average Emissions")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 6))
bottom_states.plot(kind='bar', color='green', alpha=0.7)
plt.title("Top 10 States with Lowest Yearly Average Emissions (measured in million metric tons of carbon dioxide)")
plt.xlabel("State")
plt.ylabel("Average Emissions")
plt.xticks(rotation=45)
plt.show()

This code analyzes and visualizes emissions data alongside political scores for U.S. states. It first filters out rows containing "Total" in the state name and then identifies the top and bottom 10 states based on their average emissions over time. For each of these states, it detects local minima and maxima in emissions using the `argrelextrema` function. It then plots emissions data over time, marking the local extrema points with blue (minima) and red (maxima) markers, while displaying corresponding political scores at each point. The `plot_emissions_with_extrema` function is used to generate two separate plots: one for the top 10 states with the highest emissions and another for the bottom 10 states with the lowest emissions. Additionally, the `extract_state_name` function extracts the state name from a given directory path. This comprehensive analysis helps in understanding the trends in emissions in relation to political scores over time.

In [ ]:
def extract_state_name(base_directory):
    return os.path.basename(base_directory)

In [ ]:
def analyze_and_plot_emissions(state_emissions_data, political_scores_data):
    state_emissions_data = state_emissions_data[~state_emissions_data['State'].str.contains("Total", case=False, na=False)]
    state_emissions_data = state_emissions_data[~state_emissions_data['State'].isna()]
    state_emissions_data = state_emissions_data[~state_emissions_data['State'].str.contains("Source:", na=False)]

    year_columns = [col for col in state_emissions_data.columns if isinstance(col, int)]
    top_states = state_emissions_data.set_index('State')[year_columns].mean(axis=1).nlargest(5).index
    bottom_states = state_emissions_data.set_index('State')[year_columns].mean(axis=1).nsmallest(5).index

    top_states_data = state_emissions_data[state_emissions_data['State'].isin(top_states)].set_index('State')[year_columns].T
    bottom_states_data = state_emissions_data[state_emissions_data['State'].isin(bottom_states)].set_index('State')[year_columns].T

    def find_local_extrema(data):
        extrema = {}
        for state in data.columns:
            emissions = data[state].values
            years = data.index.values
            local_min_idx = argrelextrema(emissions, np.less)[0]  # Indices of local minima
            local_max_idx = argrelextrema(emissions, np.greater)[0]  # Indices of local maxima
            extrema[state] = {
                "min": [(years[i], emissions[i]) for i in local_min_idx],  # Store (year, emission) pairs for mins
                "max": [(years[i], emissions[i]) for i in local_max_idx]   # Store (year, emission) pairs for max
            }
        return extrema

    top_extrema = find_local_extrema(top_states_data)
    bottom_extrema = find_local_extrema(bottom_states_data)

    def plot_emissions_with_extrema(data, extrema, title, scores_data):
        plt.figure(figsize=(12, 6))

        for state in data.columns:
            plt.plot(data.index, data[state], label=state)

            for x, y in extrema[state]["min"]:
                plt.scatter(x, y, color='blue', marker='v', zorder=5)
                # Convert both state names to uppercase for comparison
                score = scores_data[(scores_data['state'].str.upper() == state.upper()) & (scores_data['year'] == x)]['party_affiliation_score'].values
                if len(score) > 0:
                    # Display the Min label with party affiliation score
                    plt.text(x, y - 0.2, f"Min {score[0]:.2f}", fontsize=9, verticalalignment='bottom', horizontalalignment='center')
            for x, y in extrema[state]["max"]:
                plt.scatter(x, y, color='red', marker='^', zorder=5)
                score = scores_data[(scores_data['state'].str.upper() == state.upper()) & (scores_data['year'] == x)]['party_affiliation_score'].values
                if len(score) > 0:
                    plt.text(x, y + 0.2, f"Max {score[0]:.2f}", fontsize=9, verticalalignment='top', horizontalalignment='center')

        plt.title(title)
        plt.xlabel("Year")
        plt.ylabel("Emissions")
        plt.legend(title="States", bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside the plot
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    plot_emissions_with_extrema(top_states_data, top_extrema, "Top 5 States with Highest Emissions Over Time", political_scores_data)
    plot_emissions_with_extrema(bottom_states_data, bottom_extrema, "Top 5 States with Lowest Emissions Over Time", political_scores_data)

In [ ]:
state_emissions_data

In [ ]:
analyze_and_plot_emissions(state_emissions_data, final_political_scores_by_year)

### Demographic and Size Additions

This code performs data transformation and merging to create a comprehensive dataset for analysis. It starts by melting the `state_emissions_data` to reshape it, converting column names to lowercase and standardizing state names using a dictionary (`state_abbreviation_dict`). It then filters the data to include only valid years and merges it with `state_population_size_data` based on state and year. The `state_land_size_data` is also processed and merged to include land size information. Afterward, the political score data (`final_political_scores_by_year`) is integrated into the dataset by standardizing state names and performing another merge. The final result is a merged dataset (`final_merged_dataset`) containing emissions, population size, land size, and political scores for each state and year, ready for further analysis.

In [ ]:
state_emissions_melted = state_emissions_data.melt(id_vars=["State"],
                                                   var_name="Year",
                                                   value_name="Emissions")
state_emissions_melted.columns = state_emissions_melted.columns.str.lower()
state_emissions_melted = state_emissions_melted.applymap(lambda x: x.lower() if isinstance(x, str) else x)

state_emissions_melted["state"] = state_emissions_melted["state"].map(state_abbreviation_dict)

state_emissions_melted["year"] = state_emissions_melted["year"].astype(str)
state_emissions_melted = state_emissions_melted.dropna(subset=["year"])  # Remove NaNs
state_emissions_melted = state_emissions_melted[state_emissions_melted["year"].str.isnumeric()]  # Keep only numeric years

state_emissions_melted["year"] = state_emissions_melted["year"].astype(int)
state_population_size_data["year"] = state_population_size_data["year"].astype(int)

merged_population_and_emission_data = state_emissions_melted.merge(state_population_size_data, on=["state", "year"], how="inner")

state_land_size_data = state_land_size_data.rename(columns={"State": "state"})
state_land_size_data["state"] = state_land_size_data["state"].str.lower().map(state_abbreviation_dict)

merged_size_population_emissions_data = merged_population_and_emission_data.merge(state_land_size_data, on="state", how="inner")
merged_size_population_emissions_data = merged_size_population_emissions_data.rename(columns={"Size (Square Miles)": "size"})

final_political_scores_by_year["state"] = final_political_scores_by_year["state"].str.lower().map(state_abbreviation_dict)



all_states = final_political_scores_by_year['state'].unique()
all_years = np.arange(1970, final_political_scores_by_year['year'].max() + 1)
full_index = pd.MultiIndex.from_product([all_states, all_years], names=['state', 'year'])
final_political_scores_full = final_political_scores_by_year.set_index(['state', 'year']).reindex(full_index).reset_index()

imputer = SimpleImputer(strategy='mean')
numeric_cols = final_political_scores_full.select_dtypes(include=[np.number]).columns
final_political_scores_full[numeric_cols] = imputer.fit_transform(final_political_scores_full[numeric_cols])

final_merged_dataset = merged_size_population_emissions_data.merge(
    final_political_scores_full,
    on=["state", "year"],
    how="inner"
)

final_merged_dataset

### Bills Analysis

This code processes environmental bills and votes data across multiple directories to calculate a "Final Score" for each state and year. It begins by loading JSON files for bills and votes, normalizing and merging the data, and calculating the percentage of "yes" votes for each bill. The code identifies climate-related bills by checking for specific environmental terms in the bill description, calculates a weighted climate score based on vote percentages, and sums these scores to get the final score. The code is designed to process files for all states and years in a directory structure, aggregating the results into a final DataFrame. It also measures the execution time and saves the results to a CSV file. The final output provides a dataset of average "Final Scores" by year and state, reflecting the states' environmental legislative activity over time.

In [ ]:
def process_bills_and_votes(bill_directory, vote_directory, environmental_terms):
    def load_json_files_from_directory(directory_path):
        data_list = []
        if os.path.exists(directory_path) and os.listdir(directory_path):
            for filename in os.listdir(directory_path):
                if filename.endswith(".json"):
                    with open(os.path.join(directory_path, filename), 'r') as file:
                        data = json.load(file)
                        df = pd.json_normalize(data)
                        data_list.append(df)
            return pd.concat(data_list, ignore_index=True) if data_list else pd.DataFrame()
        return pd.DataFrame()

    bill_df = load_json_files_from_directory(bill_directory)
    if bill_df.empty:
        return 0

    bill_df.columns = [col.split('.')[-1] for col in bill_df.columns]

    vote_df = load_json_files_from_directory(vote_directory)
    if vote_df.empty:
        return 0

    vote_df.columns = [col.split('.')[-1] for col in vote_df.columns]

    merged_df = pd.merge(bill_df, vote_df, on="bill_id", how="outer")
    merged_df = merged_df.drop_duplicates(subset='bill_id', keep='first')
    merged_df['percent_yes'] = merged_df['yea'] / merged_df['total']
    merged_df = merged_df[['percent_yes', 'bill_id', 'title', 'description']].reset_index().drop(columns=['index'])

    merged_df['climate'] = merged_df['description'].apply(lambda x: 1 if any(term.lower() in str(x).lower() for term in environmental_terms) else 0)
    merged_df.fillna(0)

    climate_ratio = merged_df['climate'].sum() / len(merged_df) if len(merged_df) > 0 else 0
    merged_df['weighted_climate'] = climate_ratio * merged_df['percent_yes']
    final_score = merged_df['weighted_climate'].sum()

    return final_score

def process_all_files(base_directory, environmental_terms):
    year = os.path.basename(base_directory).split('-')[0]
    file_dict = {}

    vote_dir = os.path.join(base_directory, 'vote')
    bill_dir = os.path.join(base_directory, 'bill')

    file_dict[year] = process_bills_and_votes(bill_dir, vote_dir, environmental_terms)

    df = pd.DataFrame(list(file_dict.items()), columns=['Year', 'Final_Score'])
    return df

def process_all_states(base_directory, environmental_terms):
    results = []
    for subdir in os.listdir(base_directory):
        subdir_path = os.path.join(base_directory, subdir)
        print(subdir_path)
        if os.path.isdir(subdir_path):
            subdirectory_df = process_all_files(subdir_path, environmental_terms)
            results.append(subdirectory_df)

    state = extract_state_name(base_directory)
    final_df = pd.concat(results, ignore_index=True)
    final_df['State'] = state
    return final_df

def process_all_directories(base_directory, environmental_terms):
    all_results = []
    state_count = 1

    for state_dir in os.listdir(base_directory):
        print(f"{state_count}: Processing state: {state_dir}")
        state_count+=1
        state_dir_path = os.path.join(base_directory, state_dir)

        if os.path.isdir(state_dir_path):
            state_df = process_all_states(state_dir_path, environmental_terms)
            all_results.append(state_df)

    final_df = pd.concat(all_results, ignore_index=True)
    final_df = final_df.sort_values('Year').reset_index().drop(columns=['index'])
    final_df = final_df.groupby(['Year', 'State'])['Final_Score'].mean().reset_index()

    return final_df

def process_all_directories_timed(base_directory, environmental_terms):
    start_time = time.time()

    results_df = process_all_directories(base_directory, environmental_terms)

    end_time = time.time()
    execution_time = end_time - start_time
    results_df.to_csv('data/results/election_scores.csv')
    print(f"Execution Time: {execution_time} seconds")

    return results_df

In [ ]:
# DON'T RUN - TAKES HOURS TO TERMINATE
# results_df = process_all_directories_timed('data/bills', environmental_terms)
# results_df

This code processes the election scores data by reading a CSV file and cleaning the data by removing unnecessary columns and sorting by year and state. It then maps state abbreviations using a dictionary (`state_abbreviation_dict`). The data is grouped into four-year periods (starting from 2008), and the policy scores for each state are aggregated by summing the "Final_Score" within each year group. The resulting DataFrame is renamed to "Policy Scores" and contains aggregated scores by year group and state.

In [ ]:
election_scores_df = pd.read_csv('../data/results/election_scores.csv')
election_scores_df = election_scores_df.drop(columns=['Unnamed: 0'])
election_scores_df = election_scores_df.sort_values(by=['Year', 'State'], ascending=True)
election_scores_df['State'] = election_scores_df['State'].map(state_abbreviation_dict)

start_year = 2008
election_scores_df['Year_Group'] = (election_scores_df['Year'] - start_year) // 4 * 4 + start_year
aggregated_scores_df = election_scores_df.groupby(['Year_Group', 'State'], as_index=False)['Final_Score'].sum()
aggregated_scores_df = aggregated_scores_df.rename(columns={'Final_Score': 'Policy Scores'})
aggregated_scores_df

This code merges the `aggregated_scores_df` with `final_merged_dataset` based on matching year groups and states, using a right join to ensure all records from `final_merged_dataset` are included. It then drops the columns "State" and "Year_Group" and resets the index, cleaning up the final DataFrame. The resulting `final_df` contains the combined data with the aggregated policy scores and corresponding values from the merged dataset.

In [ ]:
final_df = aggregated_scores_df.merge(final_merged_dataset,
                                       left_on=['Year_Group', 'State'],
                                       right_on=['year', 'state'],
                                       how='right')
final_df = final_df.drop(columns=['State', 'Year_Group']).reset_index().drop(columns='index')
final_df

This code identifies the numerical and categorical columns in the `final_df` DataFrame by selecting columns with `float64` and `int64` data types for numerical columns, and excluding them for categorical columns. It then applies a K-Nearest Neighbors (KNN) imputer to fill any missing values in the numerical columns, using 5 neighbors to estimate the missing data. The imputed values are applied to a copy of `final_df`, resulting in a new DataFrame (`final_df_imputed`) with the missing numerical data filled in.

In [ ]:
numerical_columns = final_df.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = final_df.select_dtypes(exclude=['float64', 'int64']).columns

imputer = SimpleImputer(strategy='mean')

final_df_imputed = final_df.copy()
final_df_imputed[numerical_columns] = imputer.fit_transform(final_df_imputed[numerical_columns])

final_df_dropped = final_df.dropna(axis=0)

In [ ]:
final_df_imputed

#### Prepare Full Data

This code creates a copy of the `final_df_imputed` DataFrame and applies standardization to the selected columns. It first identifies the columns to normalize by excluding "state," "year," and "emissions" from the list of columns. Then, it uses the `StandardScaler` from `sklearn.preprocessing` to normalize these columns, transforming them to have a mean of 0 and a standard deviation of 1. The transformed data is assigned back to the corresponding columns in the `final_df_imputed` DataFrame, resulting in the normalized dataset.

In [ ]:
def normalize_data(df, exclude_columns=["state", "year", "emissions", "party_affiliation_score"]):
    scaler = StandardScaler()
    columns_to_normalize = [col for col in df.columns if col not in exclude_columns]
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
    return df

final_df_imputed_normalized = normalize_data(final_df_imputed)
final_df_dropped_normalized = normalize_data(final_df_dropped)

In [ ]:
state_emissions_melted = state_emissions_melted.dropna()
state_emissions_melted

In [ ]:
final_df_imputed_normalized = final_df_imputed_normalized.copy()
final_df_imputed_normalized.set_index(['state', 'year'], inplace=True)
state_emissions_melted.set_index(['state', 'year'], inplace=True)

final_df_imputed_normalized['emissions'] = state_emissions_melted['emissions']

final_df_imputed_normalized.reset_index(inplace=True)
state_emissions_melted.reset_index(inplace=True)
final_df_imputed_normalized

In [ ]:
final_df_dropped_normalized = final_df_dropped_normalized.copy()
final_df_dropped_normalized.set_index(['state', 'year'], inplace=True)
state_emissions_melted.set_index(['state', 'year'], inplace=True)

final_df_dropped_normalized['emissions'] = state_emissions_melted['emissions']

final_df_dropped_normalized.reset_index(inplace=True)
state_emissions_melted.reset_index(inplace=True)

final_df_dropped_normalized

Emissions initially declined, then rose steadily until the early 2000s, after which they began a continuous downward trend.

In [ ]:
emissions_by_year = final_df_imputed_normalized.groupby('year')['emissions'].mean().reset_index()
print(emissions_by_year)
plt.figure(figsize=(10, 6))
plt.plot(emissions_by_year['year'], emissions_by_year['emissions'], marker='o')
plt.title('Annual Emissions Over Time')
plt.xlabel('Year')
plt.ylabel('Average Emissions')
plt.grid(True)
plt.show()

In [ ]:
overall_yearly_mean = final_df_imputed_normalized.groupby('year')['emissions'].mean().rename('overall_mean')

overall_yearly_mean = overall_yearly_mean.sort_index()
yearly_changes = overall_yearly_mean.diff().dropna()
average_yearly_change = yearly_changes.abs().mean()
average_total_emissions = overall_yearly_mean.mean()
change_percentage = (average_yearly_change / average_total_emissions) * 100

print(f"Average absolute change in national emissions year-over-year: {average_yearly_change:.4f}")
print(f"Average total emissions per year: {average_total_emissions:.4f}")
print(f"Change as percentage of average total emissions: {change_percentage:.2f}%")

plt.figure(figsize=(10, 6))
plt.plot(yearly_changes.index, yearly_changes.abs(), marker='o')
plt.title('Absolute Year-over-Year Change in National Emissions')
plt.xlabel('Year')
plt.ylabel('Absolute Change in Emissions')
plt.grid(True)
plt.show()


#### Final Results

In [ ]:
# Final dataframes are:
#   1. final_df_dropped_normalized
#   2. final_df_dropped
#   3. final_df_imputed_normalized
#   4. final_df_imputed

In [ ]:
final_df = final_df_imputed.drop(columns=['Policy Scores'])
final_df_normalized = final_df_imputed_normalized.drop(columns=['Policy Scores'])

In [ ]:
df_modified_normalized = final_df_normalized.copy()
df_modified_normalized['emissions_per_capita'] = df_modified_normalized['emissions'] / df_modified_normalized['population']
df_modified_normalized['population_density'] = df_modified_normalized['population'] / df_modified_normalized['size']
df_modified_normalized = df_modified_normalized.drop(columns=['population', 'size', 'emissions'])

df_modified = final_df.copy()
df_modified['emissions_per_capita'] = df_modified['emissions'] / df_modified['population']
df_modified['population_density'] = df_modified['population'] / df_modified['size']
df_modified = df_modified.drop(columns=['population', 'size', 'emissions'])

## Adjacency Graph

In [ ]:
meta_df = (final_df_normalized[['state', 'population', 'size']]
           .groupby('state', as_index=False)
           .mean()
           .rename(columns={'size': 'area'}))     # <-- area ≡ land‑size

pop      = np.log(meta_df['population'].values)   # log‑scale as in Eq.(1)
area     = meta_df['area'].values                # already in sq‑miles
states   = meta_df['state'].tolist()
N        = len(states)

pop_z    = (pop - pop.mean()) / pop.std(ddof=0)
area_z   = (area - area.mean()) / area.std(ddof=0)


In [ ]:
from itertools import combinations
BORDERS = {  # Rook contiguity (lower‑48 + DC); AK & HI left empty
    'AL':['FL','GA','MS','TN'], 'AZ':['CA','NV','UT','NM','CO'], 'AR':['LA','MO','MS','OK','TN','TX'],
    'CA':['AZ','NV','OR'], 'CO':['AZ','KS','NM','NE','OK','UT','WY'], 'CT':['MA','NY','RI'],
    'DC':['MD','VA'], 'DE':['MD','NJ','PA'], 'FL':['AL','GA'], 'GA':['AL','FL','NC','SC','TN'],
    'HI':[], 'ID':['MT','NV','OR','UT','WA','WY'], 'IL':['IA','IN','KY','MO','WI'],
    'IN':['IL','KY','MI','OH'], 'IA':['IL','MN','MO','NE','SD','WI'], 'KS':['CO','MO','NE','OK'],
    'KY':['IL','IN','MO','OH','TN','VA','WV'], 'LA':['AR','MS','TX'], 'ME':['NH'],
    'MD':['DC','DE','PA','VA','WV'], 'MA':['CT','NH','NY','RI','VT'],
    'MI':['IN','OH','WI','MN'],  # upper‑peninsula touch via water ignored
    'MN':['IA','ND','SD','WI'], 'MS':['AL','AR','LA','TN'], 'MO':['AR','IA','IL','KS','KY','NE','OK','TN'],
    'MT':['ID','ND','SD','WY'], 'NE':['CO','IA','KS','MO','SD','WY'], 'NV':['AZ','CA','ID','OR','UT'],
    'NH':['MA','ME','VT'], 'NJ':['DE','NY','PA'], 'NM':['AZ','CO','OK','TX','UT'],
    'NY':['CT','MA','NJ','PA','VT'], 'NC':['GA','SC','TN','VA'], 'ND':['MN','MT','SD'],
    'OH':['IN','KY','MI','PA','WV'], 'OK':['AR','CO','KS','MO','NM','TX'], 'OR':['CA','ID','NV','WA'],
    'PA':['DE','MD','NJ','NY','OH','WV'], 'RI':['CT','MA'], 'SC':['GA','NC'],
    'SD':['IA','MN','MT','ND','NE','WY'], 'TN':['AL','AR','GA','KY','MO','MS','NC','VA'],
    'TX':['AR','LA','NM','OK'], 'UT':['AZ','CO','ID','NV','NM','WY'],
    'VT':['MA','NH','NY'], 'VA':['DC','KY','MD','NC','TN','WV'], 'WA':['ID','OR'],
    'WV':['KY','MD','OH','PA','VA'], 'WI':['IA','IL','MI','MN'], 'WY':['CO','ID','MT','NE','SD','UT']
}

C = np.zeros((N, N), dtype=int)
for i, s in enumerate(states):
    for nb in BORDERS.get(s, []):
        if nb in states:                           # ignore AK–HI neighbours
            j = states.index(nb)
            C[i, j] = C[j, i] = 1

In [ ]:
γ1, γ2 = 1.0, 1.0          # <<< put your grid‑search loop around these
eps    = 1e-4

W = np.full((N, N), eps)
for i, j in combinations(range(N), 2):
    if C[i, j]:
        diff_pop  = abs(pop_z[i]  - pop_z[j])
        diff_area = abs(area_z[i] - area_z[j])
        w_ij      = np.exp(-γ1*diff_pop**2 - γ2*diff_area**2)
        W[i, j] = W[j, i] = w_ij
np.fill_diagonal(W, 0.0)

# Row‑stochastic (Eq. 2)
W = W / W.sum(axis=1, keepdims=True)

In [ ]:
final_df_graph = final_df_normalized.copy(deep=True)       

state2row = dict(zip(states, range(N)))
idx_ok    = final_df_graph['state'].isin(state2row)   # drop AK / HI if absent
final_df_graph = final_df_graph.loc[idx_ok].reset_index(drop=True)

graph_cols = ['pop_g0','area_g0','pop_g1','area_g1','pop_g2','area_g2']
final_df_graph[graph_cols] = final_df_graph['state'].map(
    lambda s: pd.Series(X_graph[state2row[s]]))

print("Shape  w/out‑graph:", final_df_normalized.shape)
print("Shape with‑graph:", final_df_graph.shape)

In [ ]:
def laplacian_regression(X, y, W_rw, lamb):
    D  = np.diag(W_rw.sum(axis=1))
    L  = D - W_rw
    A  = X.T @ X + lamb * (X.T @ L @ X)
    b  = X.T @ y
    return np.linalg.solve(A, b)

latest_year = final_df['year'].max()
panel_t     = final_df[final_df['year'] == latest_year]
X_design    = panel_t[['emissions',                         #   <- lag‑10 …
                       'pop_g0','area_g0','pop_g1','area_g1','pop_g2','area_g2']].to_numpy()
y_target    = panel_t['emissions'].to_numpy()
β_hat       = laplacian_regression(X_design, y_target, W, lamb=0.3)
rmse_graph  = np.sqrt(((X_design @ β_hat - y_target) ** 2).mean())
print(f"Graph‑regularised RMSE ({latest_year:.0f}) = {rmse_graph:.2f}")

## Final Analysis

### Linear Models

#### Linear Regression

In [ ]:
def train_linear_regression_forecasting(df, window_size=3, n_splits=5, use_ridge=False, ridge_alpha=1.0):
    try:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df = df.sort_values(['state', 'year'])

        if 'state' not in df.columns or 'emissions' not in df.columns:
            print("Required columns ('state', 'emissions') are missing.")
            return

        lagged_dfs = []
        for lag in range(1, window_size + 1):
            lagged = df[['state', 'year', 'emissions']].copy()
            lagged['year'] += lag
            lagged.rename(columns={'emissions': f'emissions_t_minus_{lag}'}, inplace=True)
            lagged_dfs.append(lagged)

        for lagged in lagged_dfs:
            df = df.merge(lagged, on=['state', 'year'], how='inner')

        df = df.dropna()
        if df.empty:
            print("No data after sliding window merge.")
            return

        print(f"Shape after sliding window processing: {df.shape}")

        features = [f'emissions_t_minus_{i}' for i in range(window_size, 0, -1)] + \
                   ['population', 'size', 'party_affiliation_score']
        X = df[features]
        y = df['emissions']
        states = df['state']

        tscv = TimeSeriesSplit(n_splits=n_splits)
        r2_scores, mse_scores, explained_scores = [], [], []
        coefs_all_folds = []
        top_feature_counts = []
        top_feature_fold_counts = []  # New: track top feature per fold
        all_r2_by_state = []

        plt.figure(figsize=(10, 6))

        for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            states_test = states.iloc[test_index]

            model = Ridge(alpha=ridge_alpha, normalize=True) if use_ridge else LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            explained_var = explained_variance_score(y_test, y_pred)

            print(f"Fold {fold} — R²: {r2:.4f}, MSE: {mse:.4f}, Explained Variance: {explained_var:.4f}")
            r2_scores.append(r2)
            mse_scores.append(mse)
            explained_scores.append(explained_var)
            coefs_all_folds.append(model.coef_)

            # New: track top feature for the fold
            top_feature_fold = pd.Series(model.coef_, index=features).abs().idxmax()
            top_feature_fold_counts.append(top_feature_fold)

            plt.scatter(y_test, y_pred, label=f'Fold {fold}', alpha=0.6)

            fold_df = df.iloc[test_index].copy()
            fold_df['predicted'] = y_pred
            fold_r2 = fold_df.groupby('state').apply(lambda g: r2_score(g['emissions'], g['predicted']))
            all_r2_by_state.append(fold_r2)

            for state, group in fold_df.groupby('state'):
                if len(group) < window_size + 3:
                    continue
                X_state = group[features]
                y_state = group['emissions']
                model_state = Ridge(alpha=ridge_alpha, normalize=True) if use_ridge else LinearRegression()
                model_state.fit(X_state, y_state)
                coefs = pd.Series(model_state.coef_, index=features).abs()
                top_feature = coefs.idxmax()
                top_feature_counts.append(top_feature)

        min_val, max_val = y.min(), y.max()
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Ideal Fit')
        plt.xlabel('Actual Emissions')
        plt.ylabel('Predicted Emissions')
        plt.title('Ridge Regression' if use_ridge else 'Linear Regression')
        plt.legend(title='Fold')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print(f"\nAverage R² Score: {np.mean(r2_scores):.4f}")
        print(f"Average Mean Squared Error: {np.mean(mse_scores):.4f}")
        print(f"Average Explained Variance Score: {np.mean(explained_scores):.4f}")

        coef_matrix = np.array(coefs_all_folds)
        mean_coefs = coef_matrix.mean(axis=0)
        std_coefs = coef_matrix.std(axis=0)
        coef_df = pd.DataFrame({
            'Feature': features,
            'Mean Coefficient': mean_coefs,
            'Std Deviation': std_coefs
        }).sort_values(by='Mean Coefficient', key=abs, ascending=False)

        print("\nAverage Feature Coefficients Across All Folds:")
        print(coef_df)

        r2_by_state_all_folds = pd.concat(all_r2_by_state, axis=1).mean(axis=1)
        stratified_r2 = r2_by_state_all_folds.mean()

        print(f"\nStratified R² (average across states): {stratified_r2:.4f}")
        print(f"Highest R² State: {r2_by_state_all_folds.idxmax()} ({r2_by_state_all_folds.max():.4f})")
        print(f"Lowest R² State: {r2_by_state_all_folds.idxmin()} ({r2_by_state_all_folds.min():.4f})")

        plt.figure(figsize=(12, 6))
        r2_by_state_all_folds.sort_values(ascending=False).plot(kind='bar')
        plt.axhline(0, color='red', linestyle='--')
        plt.title('R² by State (Averaged Across Folds)')
        plt.xlabel('State')
        plt.ylabel('R² Score')
        plt.tight_layout()
        plt.show()

        percent_party_top = (sum(f == 'party_affiliation_score' for f in top_feature_counts) / len(top_feature_counts)) * 100
        print(f"\nPercentage of state models (across all folds) where 'party_affiliation_score' was the top predictor: {percent_party_top:.2f}%")

        percent_party_top_fold = (sum(f == 'party_affiliation_score' for f in top_feature_fold_counts) / len(top_feature_fold_counts)) * 100
        print(f"Percentage of folds where 'party_affiliation_score' was the top predictor: {percent_party_top_fold:.2f}%")

        return (
            r2_scores,
            mse_scores,
            explained_scores,
            coef_df,
            stratified_r2,
            r2_by_state_all_folds.sort_values(ascending=False),
            percent_party_top,
            percent_party_top_fold
        )

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


##### Normal

In [ ]:
r2_scores, mse_scores, explained_scores, coef_df, stratified_r2, r2_by_state, percent_party_top, percent_party_top_fold = train_linear_regression_forecasting(final_df_normalized, window_size=10)

In [ ]:
r2_scores, mse_scores, explained_scores, coef_df, stratified_r2, r2_by_state, percent_party_top, percent_party_top_fold = train_linear_regression_forecasting(final_df_normalized, window_size=5)

In [ ]:
r2_scores, mse_scores, explained_scores, coef_df, stratified_r2, r2_by_state, percent_party_top, percent_party_top_fold = train_linear_regression_forecasting(
    final_df_graph,
    window_size=10,
    extra_graph_features=['pop_g0','area_g0','pop_g1','area_g1','pop_g2','area_g2']
) 


##### Ridge

In [ ]:
train_linear_regression_forecasting(final_df_normalized, window_size=10, n_splits=5, use_ridge=True, ridge_alpha=0.5)

In [ ]:
train_linear_regression_forecasting(final_df_normalized, window_size=5, n_splits=5, use_ridge=True, ridge_alpha=0.5)

In [ ]:
graph_cols = ['pop_g0','area_g0','pop_g1','area_g1','pop_g2','area_g2']
ridge_graph_10 = train_linear_regression_forecasting(
    final_df_graph,           
    window_size=10,
    n_splits=5,
    use_ridge=True,
    ridge_alpha=0.5,
    extra_graph_features=graph_cols  
)

In [ ]:
ridge_graph_5 = train_linear_regression_forecasting(
    final_df_graph,           
    window_size=5,
    n_splits=5,
    use_ridge=True,
    ridge_alpha=0.5,
    extra_graph_features=graph_cols  
)

#### Clustering

In [ ]:
final_df_normalized_copy = final_df_normalized

final_df_normalized_copy['emissions_change'] = (
    final_df_normalized_copy
    .sort_values(['state', 'year'])
    .groupby('state')['emissions']
    .diff()
)

state_means = final_df_normalized_copy.drop(columns=['year']).groupby('state').mean().reset_index()
r2_df = r2_by_state.reset_index()
r2_df.columns = ['state', 'r2_score']
merged_df = state_means.merge(r2_df, on='state', how='inner')
merged_df

In [ ]:
def analyze_r2_clusters_from_merged(merged_df, max_k=10):
    try:
        clustering_features = ['r2_score', 'party_affiliation_score', 'emissions_change', 'population', 'size', 'emissions']

        for col in clustering_features:
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

        merged_df[clustering_features] = merged_df[clustering_features].fillna(
            merged_df[clustering_features].mean()
        )

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(merged_df[clustering_features])

        sse = []
        K_range = range(1, min(len(X_scaled), max_k) + 1)
        for k in K_range:
            km = KMeans(n_clusters=k, random_state=42, n_init=10)
            km.fit(X_scaled)
            sse.append(km.inertia_)

        kn = KneeLocator(K_range, sse, curve='convex', direction='decreasing')
        optimal_k = kn.elbow if kn.elbow is not None else 3  # fallback if not found

        plt.figure(figsize=(6, 4))
        plt.plot(K_range, sse, marker='o')
        if kn.elbow:
            plt.axvline(x=kn.elbow, color='red', linestyle='--', label=f'Elbow at k={kn.elbow}')
            plt.legend()
        plt.title('Elbow Method to Determine Optimal Clusters')
        plt.xlabel('Number of Clusters')
        plt.ylabel('SSE (Inertia)')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print(f"✅ Optimal number of clusters selected: {optimal_k}")

        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        merged_df['cluster'] = kmeans.fit_predict(X_scaled)

        cluster_summary = merged_df.groupby('cluster')[clustering_features].mean().round(3)
        feature_ranges = cluster_summary.max() - cluster_summary.min()
        top_discriminative_features = feature_ranges.sort_values(ascending=False)

        plt.figure(figsize=(8, 6))
        sns.scatterplot(
            data=merged_df,
            x='party_affiliation_score',
            y='r2_score',
            hue='cluster',
            palette='viridis',
            s=100
        )
        plt.axhline(0, color='red', linestyle='--')
        plt.title('State R² vs. Party Affiliation Score (Clustered)')
        plt.xlabel('Party Affiliation Score')
        plt.ylabel('R² Score')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print("\nCluster Summary:")
        print(cluster_summary)

        print("\nTop Discriminative Features Across Clusters:")
        print(top_discriminative_features)

        return merged_df, cluster_summary, top_discriminative_features

    except Exception as e:
        print(f"\n❌ An error occurred in clustering: {e}")
        return None

In [ ]:
clustered_df, summary_df, top_features = analyze_r2_clusters_from_merged(merged_df)

#### Logistic Regression

In [ ]:
final_df_normalized_tmp = final_df_normalized_copy
final_df_normalized_tmp['political_party'] = final_df_normalized['party_affiliation_score'].apply(lambda x: 'D' if x > 0 else 'R')
scaler = StandardScaler()
final_df_normalized_tmp['emissions'] = scaler.fit_transform(final_df_normalized_tmp[['emissions']])
final_df_normalized_tmp

In [ ]:
def average_change_by_party(df, value_column='emissions', party_column='political_party'):
    df = df.sort_values(['state', 'year'])
    df['value_diff'] = df.groupby('state')[value_column].diff()
    state_avg_change = df.groupby('state')['value_diff'].mean().reset_index()
    party_map = df.drop_duplicates('state')[['state', party_column]]
    merged = state_avg_change.merge(party_map, on='state', how='left')
    result = merged.groupby(party_column)['value_diff'].mean().reset_index().rename(columns={'value_diff': 'avg_change'})
    return result

average_change_by_party(final_df_normalized_tmp)

In [ ]:
counts = final_df_normalized_tmp['political_party'].value_counts(normalize=True) * 100
print(counts)

In [ ]:
def train_logistic_regression_forecasting(
    df,
    window_size=3,
    n_splits=5,
    penalty='l2',
    C=1.0,
    exclude_demo_features=False,
    shuffle_labels=True
):
    try:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df = df.sort_values(['state', 'year'])

        if 'state' not in df.columns or 'political_party' not in df.columns:
            print("Required columns ('state', 'political_party') are missing.")
            return

        lagged_dfs = []
        for lag in range(1, window_size + 1):
            lagged = df[['state', 'year', 'emissions']].copy()
            lagged['year'] += lag
            lagged.rename(columns={'emissions': f'emissions_t_minus_{lag}'}, inplace=True)
            lagged_dfs.append(lagged)

        for lagged in lagged_dfs:
            df = df.merge(lagged, on=['state', 'year'], how='inner')

        df = df.dropna()
        if df.empty:
            print("No data after sliding window merge.")
            return

        print(f"Shape after sliding window processing: {df.shape}")

        base_features = [f'emissions_t_minus_{i}' for i in range(window_size, 0, -1)]
        extra_features = ['population', 'size']
        features = base_features + ([] if exclude_demo_features else extra_features)

        X = df[features]
        y = df['political_party'].copy()
        states = df['state']

        if shuffle_labels:
            print("Shuffling political_party labels to test against random baseline...")
            y = y.sample(frac=1.0, random_state=42).reset_index(drop=True)

        tscv = TimeSeriesSplit(n_splits=n_splits)
        accuracy_scores = []
        precision_scores = []
        recall_scores = []
        f1_scores = []
        coefs_all_folds = []
        top_feature_counts = []

        plt.figure(figsize=(10, 6))

        for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            states_test = states.iloc[test_index]

            if len(np.unique(y_train)) < 2:
                print(f"Fold {fold} skipped — only one class ({np.unique(y_train)[0]}) present in training data.")
                continue

            model = LogisticRegression(penalty=penalty, C=C, max_iter=1000)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred, pos_label='R', zero_division=0)
            rec = recall_score(y_test, y_pred, pos_label='R', zero_division=0)
            f1 = f1_score(y_test, y_pred, pos_label='R', zero_division=0)

            accuracy_scores.append(acc)
            precision_scores.append(prec)
            recall_scores.append(rec)
            f1_scores.append(f1)
            coefs_all_folds.append(model.coef_[0])

            print(f"Fold {fold} — Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
            plt.scatter(range(len(y_test)), y_pred, alpha=0.6, label=f'Fold {fold}')

            fold_df = df.iloc[test_index].copy()
            for state, group in fold_df.groupby('state'):
                if len(group) < window_size + 3:
                    continue
                X_state = group[features]
                y_state = y.iloc[group.index]
                if len(np.unique(y_state)) < 2:
                    continue
                model_state = LogisticRegression(penalty=penalty, C=C, max_iter=1000)
                model_state.fit(X_state, y_state)
                coefs = pd.Series(model_state.coef_[0], index=features).abs()
                top_feature = coefs.idxmax()
                top_feature_counts.append(top_feature)

            if fold == n_splits:
                final_test_df = pd.DataFrame({
                    'state': states_test.values,
                    'actual': y_test.values,
                    'predicted': y_pred
                })

        plt.xlabel('Sample Index')
        plt.ylabel('Predicted Political Party')
        plt.title('Logistic Regression Predictions by Fold')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        if not accuracy_scores:
            print("\nNo valid folds with both classes. Cannot compute overall metrics.")
            return None

        print(f"\nAverage Accuracy Across Folds:  {np.mean(accuracy_scores):.4f}")
        print(f"Average Precision (R):          {np.mean(precision_scores):.4f}")
        print(f"Average Recall (R):             {np.mean(recall_scores):.4f}")
        print(f"Average F1 Score (R):           {np.mean(f1_scores):.4f}")

        coef_matrix = np.array(coefs_all_folds)
        mean_coefs = coef_matrix.mean(axis=0)
        std_coefs = coef_matrix.std(axis=0)

        coef_df = pd.DataFrame({
            'Feature': features,
            'Mean Coefficient': mean_coefs,
            'Std Deviation': std_coefs
        }).sort_values(by='Mean Coefficient', key=abs, ascending=False)

        print("\nAverage Feature Coefficients Across All Folds:")
        print(coef_df)

        non_lag_df = coef_df[~coef_df['Feature'].str.startswith('emissions_t_minus_')]
        print("\nNon-Lag Feature Coefficients (Excludes emissions_t_minus_X):")
        print(non_lag_df)

        accuracy_by_state = final_test_df.groupby('state').apply(lambda g: accuracy_score(g['actual'], g['predicted']))
        stratified_accuracy = accuracy_by_state.mean()

        print(f"\nStratified Accuracy (average across states): {stratified_accuracy:.4f}")
        print(f"Highest Accuracy State: {accuracy_by_state.idxmax()} ({accuracy_by_state.max():.4f})")
        print(f"Lowest Accuracy State: {accuracy_by_state.idxmin()} ({accuracy_by_state.min():.4f})")

        plt.figure(figsize=(12, 6))
        accuracy_by_state.sort_values(ascending=False).plot(kind='bar')
        plt.axhline(0.5, color='red', linestyle='--')
        plt.title('Accuracy by State (Last Fold)')
        plt.xlabel('State')
        plt.ylabel('Accuracy')
        plt.tight_layout()
        plt.show()

        total_state_models = len(top_feature_counts)
        top_feature_counts_series = pd.Series(top_feature_counts)
        top_feature_counts_percent = top_feature_counts_series.value_counts(normalize=True) * 100

        print("\nTop Feature Frequency (Percent of States Where Each Feature Was Most Predictive):")
        print(top_feature_counts_percent)

        return (
            accuracy_scores,
            coef_df,
            non_lag_df,
            stratified_accuracy,
            accuracy_by_state.sort_values(ascending=False),
            top_feature_counts_percent
        )

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [ ]:
train_logistic_regression_forecasting(final_df_normalized_tmp, window_size=10)

In [ ]:
train_logistic_regression_forecasting(final_df_normalized_tmp, window_size=5)

#### Engineered Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def train_feature_engineered_lr(df, window_size=3, n_splits=5, use_ridge=False, ridge_alpha=1.0):
    try:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df = df.sort_values(['state', 'year'])

        if 'state' not in df.columns or 'emissions_per_capita' not in df.columns:
            print("Required columns ('state', 'emissions_per_capita') are missing.")
            return

        # Create lagged emissions_per_capita features
        lagged_dfs = []
        for lag in range(1, window_size + 1):
            lagged = df[['state', 'year', 'emissions_per_capita']].copy()
            lagged['year'] += lag
            lagged.rename(columns={'emissions_per_capita': f'emissions_t_minus_{lag}'}, inplace=True)
            lagged_dfs.append(lagged)

        for lagged in lagged_dfs:
            df = df.merge(lagged, on=['state', 'year'], how='inner')

        df = df.dropna()
        if df.empty:
            print("No data after sliding window merge.")
            return

        print(f"Shape after sliding window processing: {df.shape}")

        features = [f'emissions_t_minus_{i}' for i in range(window_size, 0, -1)] + ['population_density']
        X = df[features]
        y = df['party_affiliation_score']
        states = df['state']

        tscv = TimeSeriesSplit(n_splits=n_splits)
        r2_scores, mse_scores, explained_scores = [], [], []
        coefs_all_folds, top_feature_counts = [], []

        plt.figure(figsize=(10, 6))

        for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            states_test = states.iloc[test_idx]

            model = Ridge(alpha=ridge_alpha) if use_ridge else LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            explained_var = explained_variance_score(y_test, y_pred)

            print(f"Fold {fold} — R²: {r2:.4f}, MSE: {mse:.4f}, Explained Variance: {explained_var:.4f}")
            r2_scores.append(r2)
            mse_scores.append(mse)
            explained_scores.append(explained_var)
            coefs_all_folds.append(model.coef_)

            plt.scatter(y_test, y_pred, label=f'Fold {fold}', alpha=0.6)

            # Feature importance tracking per state
            fold_df = df.iloc[test_idx].copy()
            for state, group in fold_df.groupby('state'):
                if len(group) < window_size + 3:
                    continue
                X_state = group[features]
                y_state = group['party_affiliation_score']
                model_state = Ridge(alpha=ridge_alpha) if use_ridge else LinearRegression()
                model_state.fit(X_state, y_state)
                coefs = pd.Series(model_state.coef_, index=features).abs()
                top_feature = coefs.idxmax()
                top_feature_counts.append(top_feature)

            if fold == n_splits:
                final_test_df = pd.DataFrame({
                    'state': states_test.values,
                    'actual': y_test.values,
                    'predicted': y_pred
                })

        min_val, max_val = min(y.min(), y.max()), max(y.min(), y.max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Ideal Fit')
        plt.xlabel('Actual Party Affiliation Score')
        plt.ylabel('Predicted Score')
        plt.title('Feature Engineered Linear Regression' + (' (Ridge)' if use_ridge else ''))
        plt.legend(title='Fold')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print(f"\nAverage R² Score: {np.mean(r2_scores):.4f}")
        print(f"Average Mean Squared Error: {np.mean(mse_scores):.4f}")
        print(f"Average Explained Variance Score: {np.mean(explained_scores):.4f}")

        coef_matrix = np.array(coefs_all_folds)
        mean_coefs = coef_matrix.mean(axis=0)
        std_coefs = coef_matrix.std(axis=0)

        coef_df = pd.DataFrame({
            'Feature': features,
            'Mean Coefficient': mean_coefs,
            'Std Deviation': std_coefs
        }).sort_values(by='Mean Coefficient', key=abs, ascending=False)

        print("\nAverage Feature Coefficients Across All Folds:")
        print(coef_df)

        r2_by_state = final_test_df.groupby('state').apply(lambda g: r2_score(g['actual'], g['predicted']))
        stratified_r2 = r2_by_state.mean()

        print(f"\nStratified R² (average across states): {stratified_r2:.4f}")
        print(f"Highest R² State: {r2_by_state.idxmax()} ({r2_by_state.max():.4f})")
        print(f"Lowest R² State: {r2_by_state.idxmin()} ({r2_by_state.min():.4f})")

        plt.figure(figsize=(12, 6))
        r2_by_state.sort_values(ascending=False).plot(kind='bar')
        plt.axhline(0, color='red', linestyle='--')
        plt.title('R² by State (Last Fold)')
        plt.xlabel('State')
        plt.ylabel('R² Score')
        plt.tight_layout()
        plt.show()

        total_state_models = len(top_feature_counts)
        top_emissions_pct = (sum(f.startswith('emissions') for f in top_feature_counts) / total_state_models) * 100

        print(f"\nPercentage of state models where an 'emissions_t_minus_X' feature was most predictive: {top_emissions_pct:.2f}%")

        return r2_scores, mse_scores, explained_scores, coef_df, stratified_r2, r2_by_state.sort_values(ascending=False), top_emissions_pct

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


In [ ]:
results = train_feature_engineered_lr(df_modified_normalized)

### Random Forest

In [ ]:
final_df_tmp = final_df
final_df_tmp['political_party'] = final_df['party_affiliation_score'].apply(lambda x: 'D' if x > 0 else 'R')
final_df_tmp

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest_forecasting(df, window_size=5, n_splits=5, n_estimators=100, extra_graph_features=None):
    extra_graph_features = extra_graph_features or []
    try:
        df['year'] = pd.to_numeric(df['year'], errors='coerce')
        df = df.sort_values(['state', 'year'])

        if 'state' not in df.columns or 'emissions' not in df.columns:
            print("Required columns ('state', 'emissions') are missing.")
            return

        lagged_dfs = []
        for lag in range(1, window_size + 1):
            lagged = df[['state', 'year', 'emissions']].copy()
            lagged['year'] += lag
            lagged.rename(columns={'emissions': f'emissions_t_minus_{lag}'}, inplace=True)
            lagged_dfs.append(lagged)

        for lagged in lagged_dfs:
            df = df.merge(lagged, on=['state', 'year'], how='inner')

        df = df.dropna()
        if df.empty:
            print("No data after sliding window merge.")
            return

        print(f"Shape after sliding window processing: {df.shape}")

        features = [f'emissions_t_minus_{i}' for i in range(window_size, 0, -1)] + \
                   ['population', 'size', 'party_affiliation_score'] + extra_graph_features   
        X = df[features]
        y = df['emissions']
        states = df['state']

        tscv = TimeSeriesSplit(n_splits=n_splits)
        r2_scores, mse_scores, explained_scores = [], [], []
        feature_importance_all_folds = []
        top_feature_counts = []
        all_r2_by_state = []

        plt.figure(figsize=(10, 6))

        for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            states_test = states.iloc[test_index]

            model = RandomForestRegressor(n_estimators=n_estimators)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            explained_var = explained_variance_score(y_test, y_pred)

            print(f"Fold {fold} — R²: {r2:.4f}, MSE: {mse:.4f}, Explained Variance: {explained_var:.4f}")
            r2_scores.append(r2)
            mse_scores.append(mse)
            explained_scores.append(explained_var)
            feature_importance_all_folds.append(model.feature_importances_)

            top_feature_fold = pd.Series(model.feature_importances_, index=features).idxmax()
            top_feature_counts.append(top_feature_fold)

            plt.scatter(y_test, y_pred, label=f'Fold {fold}', alpha=0.6)

            fold_df = df.iloc[test_index].copy()
            fold_df['predicted'] = y_pred
            fold_r2 = fold_df.groupby('state').apply(lambda g: r2_score(g['emissions'], g['predicted']))
            all_r2_by_state.append(fold_r2)

        min_val, max_val = y.min(), y.max()
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Ideal Fit')
        plt.xlabel('Actual Emissions')
        plt.ylabel('Predicted Emissions')
        plt.title('Random Forest Regression')
        plt.legend(title='Fold')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print(f"\nAverage R² Score: {np.mean(r2_scores):.4f}")
        print(f"Average Mean Squared Error: {np.mean(mse_scores):.4f}")
        print(f"Average Explained Variance Score: {np.mean(explained_scores):.4f}")

        # Feature importances
        importances_matrix = np.array(feature_importance_all_folds)
        mean_importances = importances_matrix.mean(axis=0)
        std_importances = importances_matrix.std(axis=0)
        importance_df = pd.DataFrame({
            'Feature': features,
            'Mean Importance': mean_importances,
            'Std Deviation': std_importances
        }).sort_values(by='Mean Importance', ascending=False)

        print("\nAverage Feature Importances Across All Folds:")
        print(importance_df)

        # Stratified R² by state
        r2_by_state_all_folds = pd.concat(all_r2_by_state, axis=1).mean(axis=1)
        stratified_r2 = r2_by_state_all_folds.mean()

        print(f"\nStratified R² (average across states): {stratified_r2:.4f}")
        print(f"Highest R² State: {r2_by_state_all_folds.idxmax()} ({r2_by_state_all_folds.max():.4f})")
        print(f"Lowest R² State: {r2_by_state_all_folds.idxmin()} ({r2_by_state_all_folds.min():.4f})")

        plt.figure(figsize=(12, 6))
        r2_by_state_all_folds.sort_values(ascending=False).plot(kind='bar')
        plt.axhline(0, color='red', linestyle='--')
        plt.title('R² by State (Averaged Across Folds)')
        plt.xlabel('State')
        plt.ylabel('R² Score')
        plt.tight_layout()
        plt.show()

        percent_party_top = (sum(f == 'party_affiliation_score' for f in top_feature_counts) / len(top_feature_counts)) * 100
        print(f"\nPercentage of folds where 'party_affiliation_score' was the most important feature: {percent_party_top:.2f}%")

        return (
            r2_scores,
            mse_scores,
            explained_scores,
            importance_df,
            stratified_r2,
            r2_by_state_all_folds.sort_values(ascending=False),
            percent_party_top
        )

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


In [ ]:
train_random_forest_forecasting(final_df_tmp, window_size=10)

In [ ]:
train_random_forest_forecasting(final_df_tmp, window_size=5)

In [ ]:
train_random_forest_forecasting(final_df_tmp, window_size=10, extra_graph_features = graph_cols)

In [ ]:
train_random_forest_forecasting(final_df_tmp, window_size=5, extra_graph_features = graph_cols)